## Identify the class for each item, using the hint

In [1]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU
from keras.optimizers import SGD

In [2]:
seqdataIn = pd.read_json('../data/seq-public.json', orient='records')
print(seqdataIn)

                  stem               options            category    id
0           7,9,-1,5,?        [4, 2, -1, -3]            sequence     0
1        3,2,5/3,3/2,?  [1/4, 7/5, 3/4, 2/5]            sequence     1
2           1,2,5,26,?    [34, 841, 677, 37]            sequence     2
3            2,12,30,?      [50, 65, 75, 56]            sequence     3
4        2,1,2/3,1/2,?  [3/4, 1/4, 2/5, 5/6]            sequence     4
...                ...                   ...                 ...   ...
1071  20 22 25 30 37 ?                    []  sequence-reasoning  1090
1072        0 1 3 10 ?                    []  sequence-reasoning  1091
1073       5 15 10 215                    []  sequence-reasoning  1092
1074        1 2 5 29 ?    [34, 841, 866, 37]  sequence-reasoning  1093
1075         2 12 30 ?      [50, 65, 75, 56]  sequence-reasoning  1094

[1076 rows x 4 columns]


In [3]:
seqDataAns =pd.read_json('../data/seq-public.answer.json',orient='index')
print(seqDataAns)

     answer                                               hint
0       [4]      A<sub>n+1</sub>=2<sup>5-n</sup>-A<sub>n</sub>
1       [2]  3/1, 4/2, 5/3, 6/4\n\nA<sub>n+1</sub>=A<sub>n<...
2       [3]        A<sub>n+1</sub>=A<sub>n</sub><sup>2</sup>+1
3       [4]                           A<sub>n</sub>=2n\*(2n-1)
4       [3]             4/2, 4/4, 4/6, 4/8\n\nA<sub>n</sub>=2n
...     ...                                                ...
1090     48                A<sub>n+1</sub>-A<sub>n</sub>=P(n);
1091    102           A<sub>n\*2</sub>=A<sub>n\*2-1</sub>^2+2;
1092   -115   A<sub>n</sub>=A<sub>n-2</sub>^2-A<sub>n-1</sub>;
1093    [3]  A<sub>n</sub>=A<sub>n-1</sub>^2+A<sub>n-2</sub...
1094    [4]                      A<sub>n</sub>=(2\*n-1)\*2\*n;

[1076 rows x 2 columns]


In [4]:
def hint2txt(h):
    return h.replace('<sub>','_{').replace('</sub>','}').replace('<sup>','^{').replace('</sup>','}').replace('\n',' | ').replace('\*','*')
seqDataAns['hint'] = seqDataAns['hint'].map(lambda x: hint2txt(x))
print(seqDataAns['hint'])

0                                   A_{n+1}=2^{5-n}-A_{n}
1       3/1, 4/2, 5/3, 6/4 |  | A_{n+1}=A_{n}+1 |  | B...
2                                     A_{n+1}=A_{n}^{2}+1
3                                         A_{n}=2n*(2n-1)
4                        4/2, 4/4, 4/6, 4/8 |  | A_{n}=2n
                              ...                        
1090                                  A_{n+1}-A_{n}=P(n);
1091                               A_{n*2}=A_{n*2-1}^2+2;
1092                             A_{n}=A_{n-2}^2-A_{n-1};
1093                           A_{n}=A_{n-1}^2+A_{n-2}^2;
1094                                   A_{n}=(2*n-1)*2*n;
Name: hint, Length: 1076, dtype: object


In [5]:
seqDataAns['hint1'] = [item.split('=') for item in seqDataAns['hint']] #split the hint by '=', to search item
                                                                        #in second term (after =)

In [6]:
seqDataAns['classes'] = 0

In [7]:
for i in range(len(seqDataAns)):
    if i in [82,  257, 298, 306, 451, 452, 463, 464,468, 476,495, 498,499, 500,504,505,623,624,626,627,628]:
        continue
        
    if seqDataAns['classes'][i] == 0:
        y = False
        z = False
        if len(seqDataAns['hint1'][i])>1:
            y = re.search('A_{', seqDataAns['hint1'][i][1])
            z = re.search('(?<!_).*?n', seqDataAns['hint1'][i][1])

        if (y) and not(z):
            seqDataAns.loc[i, 'classes'] = 1   #recursive A_{n}
            
        elif not(y) and (z):
            seqDataAns.loc[i, 'classes'] = 2   #just the position n
        
        elif not(y) and not(z):
            seqDataAns.loc[i, 'classes'] = 0   #none, or errors
            
        else:
            seqDataAns.loc[i, 'classes'] = 3   #both
            
        

In [23]:
seqDataAns[seqDataAns['classes'] == 0][100:140]

,answer,hint,hint1,classes
522,[3],A_{n}%3=0,"[A_{n}%3, 0]",0
532,[2],A_{n+1}-A_{n}=13,"[A_{n+1}-A_{n}, 13]",0
548,[4],A_{2n} / A_{2n-1} = 3,"[A_{2n} / A_{2n-1} , 3]",0
607,[4],An/An+1=2/3,"[An/An+1, 2/3]",0
617,[2],An+1/An=3,"[An+1/An, 3]",0
622,38,,[],0
625,60,,[],0
629,14,number are odd number,[number are odd number],0
639,[3],,[],0
642,[1],,[],0


In [24]:
seqdataIn[seqdataIn['id'] == 522]

,stem,options,category,id
508,"15,18,54,?,210","[106, 107, 123, 112]",sequence,522


In [12]:
seqDataAns.classes.value_counts()

3    511
2    380
0    184
1      1
Name: classes, dtype: int64

    answer                                       hint  \
954    [1]  A_{1}+A_{9}=A_{2}+A_{8}+6=A_{3}+A_{7}+12;   

                                             hint1  classes  
954  [A_{1}+A_{9}, A_{2}+A_{8}+6, A_{3}+A_{7}+12;]        1  


## Sequence parsing

In [19]:
sequences = seqdataIn['stem']
print(sequences)

0             7,9,-1,5,?
1          3,2,5/3,3/2,?
2             1,2,5,26,?
3              2,12,30,?
4          2,1,2/3,1/2,?
              ...       
1071    20 22 25 30 37 ?
1072          0 1 3 10 ?
1073         5 15 10 215
1074          1 2 5 29 ?
1075           2 12 30 ?
Name: stem, Length: 1076, dtype: object


In [20]:
#cleans up sequences
def txt2seq(seq):
    s2 = {}
    for i,s in seq.items():
        si = re.sub("\(\s?\)","?",s)       #replace parenthesis with question mark
        a = re.split(',|\s',si)      #remove any delimiters
        a = [e for e in a if e != ""]  #remove empty strings
        s2[i] = a
    return s2
split_seq = txt2seq(sequences)
print("Split: " + str(split_seq))

Split: {0: ['7', '9', '-1', '5', '?'], 1: ['3', '2', '5/3', '3/2', '?'], 2: ['1', '2', '5', '26', '?'], 3: ['2', '12', '30', '?'], 4: ['2', '1', '2/3', '1/2', '?'], 5: ['4', '2', '2', '3', '6', '?'], 6: ['1', '7', '8', '57', '?'], 7: ['4', '12', '8', '10', '?'], 8: ['1/2', '1', '1', '?', '9/11', '11/13'], 9: ['95', '88', '71', '61', '50', '?'], 10: ['2', '6', '13', '39', '15', '45', '23', '?'], 11: ['1', '3', '3', '5', '7', '9', '13', '15', '?', '?'], 12: ['1', '2', '8', '28', '?'], 13: ['0', '4', '18', '?', '100'], 14: ['1', '1', '2', '2', '3', '4', '3', '5', '?'], 15: ['1', '52', '313', '174', '?'], 16: ['5', '15', '10', '215', '?'], 17: ['-7', '0', '1', '2', '9', '?'], 18: ['0', '1', '3', '10', '?'], 19: ['9/2', '14', '65/2', '?', '217/2'], 20: ['1', '1', '2', '6', '24', '?'], 21: ['3', '4', '8', '24', '88', '?'], 22: ['20', '22', '25', '30', '37', '?'], 23: ['1/9', '2/27', '1/27'], 24: ['√2', '3', '√28', '√65', '?'], 25: ['1', '2', '4', '8', '16', '?'], 26: ['2', '1', '2/3', '1/2',

In [21]:
#checks for valid sequences (all numbers or fractions excluding ?)
def findBadSeq(seq):
    badSeqs = {}
    for a in seq.keys():     #check each sequence
        s = seq[a]
        for i in s:   #check each item (assuming sequence is already split)
            if(re.match(r'[^0-9\/\?\-]',i)):    #check if any words or weird characters in the sequence
                badSeqs[a] = s
                break
    return badSeqs

#remove bad sequences from good sequences
def remBadSeq(seq):
    badSeqInd = list(findBadSeq(seq).keys())
    goodSeq = {}
    for i in seq.keys():
        if i not in badSeqInd:     #want to keep the indexes to get the answer for it
            goodSeq[i] = seq[i]
    return goodSeq


In [22]:
# show bad sequences removed
bs = findBadSeq(split_seq)
for k,v in bs.items():
    print(str(k) + ":" + str(v))
print("# Bad sequences = " + str(len(bs)) + "/" + str(len(sequences)))

24:['√2', '3', '√28', '√65', '?']
85:['A', 'F', 'H', 'K', 'N', '?', '?']
86:['A', 'D', 'I', 'P', 'Y', 'CF', 'DI', 'FD', '?']
118:['+1', '-1', '1', '-1', '?']
161:['19/13', '1', '19/13', '10/22', '?', 'a.7/24;b.7/25;c.5/26;d.7/26;']
297:['√5', '√55', '11√5', '11√55', '?']
367:['0', '3', '1', '6', '√2', '12', '?', '?', '2', '48']
389:['1/2', '√3/2', '√3/3', '?']
411:['(√5-1)/4', '1/3', '(√3', '-1)/2', '√2', '-1', '?']
470:['A', 'C', 'F', 'J', 'O', '?', 'What', 'letter', 'comes', 'next?']
487:['72', '(68)', '41', '28', '(98)', '16', '34', '?', '56']
489:['4342', '(3176)', '1726', '7995', '(7516)', '2162', '8418', '?', '1725']
491:['-3', '+6', '+2', '-3', '+7', '-12', '?']
608:['961', '(25)', '432', '932', '?', '731']
609:['16', '(96)', '12', '10', '?', '7.5']
610:['The', 'following', 'series', 'of', 'numbers', 'contains', 'one', 'number', 'that', 'does', 'not', 'fit', 'the', 'pattern', 'set', 'by', 'the', 'others.', 'What', 'number', 'does', 'not', 'fit?', '3', '5', '7', '11', '14', '17']

In [23]:
#get only good sequences
gs = remBadSeq(split_seq)
for k,v in gs.items():
    print(str(k) + ":" + str(v))
print("# Good sequences = " + str(len(gs)) + "/" + str(len(sequences)))

0:['7', '9', '-1', '5', '?']
1:['3', '2', '5/3', '3/2', '?']
2:['1', '2', '5', '26', '?']
3:['2', '12', '30', '?']
4:['2', '1', '2/3', '1/2', '?']
5:['4', '2', '2', '3', '6', '?']
6:['1', '7', '8', '57', '?']
7:['4', '12', '8', '10', '?']
8:['1/2', '1', '1', '?', '9/11', '11/13']
9:['95', '88', '71', '61', '50', '?']
10:['2', '6', '13', '39', '15', '45', '23', '?']
11:['1', '3', '3', '5', '7', '9', '13', '15', '?', '?']
12:['1', '2', '8', '28', '?']
13:['0', '4', '18', '?', '100']
14:['1', '1', '2', '2', '3', '4', '3', '5', '?']
15:['1', '52', '313', '174', '?']
16:['5', '15', '10', '215', '?']
17:['-7', '0', '1', '2', '9', '?']
18:['0', '1', '3', '10', '?']
19:['9/2', '14', '65/2', '?', '217/2']
20:['1', '1', '2', '6', '24', '?']
21:['3', '4', '8', '24', '88', '?']
22:['20', '22', '25', '30', '37', '?']
23:['1/9', '2/27', '1/27']
25:['1', '2', '4', '8', '16', '?']
26:['2', '1', '2/3', '1/2', '?']
27:['1', '1', '3', '7', '17', '41', '?']
28:['5/2', '5', '25/2', '75/2', '?']
29:['6', '1

1005:['10', '9', '17', '50', '?']
1006:['11', '34', '75', '?', '235']
1007:['2', '3', '5', '7', '11', '13', '?']
1008:['0', '4', '18', '48', '?']
1009:['0', '9', '26', '85', '124', '?']
1010:['3.02', '4.03', '3.05', '9.08', '?']
1011:['1', '2', '8', '28', '?']
1012:['290', '288', '?', '294', '279', '301', '275']
1013:['0', '4', '18', '?', '100']
1014:['2', '1', '2/3', '1/2', '?']
1015:['4', '5', '8', '10', '?']
1016:['95', '88', '80', '71', '61', '50', '?']
1017:['-2', '1', '7', '16', '?', '43']
1018:['?', '36', '19', '10', '5', '2']
1019:['5', '17', '21', '25', '?']
1020:['3', '6', '21', '60', '?']
1021:['1', '1', '3', '7', '17', '41', '?']
1022:['1/6', '2/3', '3/2', '8/3', '?']
1023:['3', '2', '5/3', '3/2', '?']
1024:['27', '16', '5', '?', '1/7']
1025:['0', '1', '1', '2', '4', '7', '13', '?']
1026:['1', '0', '-1', '-2', '?']
1027:['-1', '0', '27', '?']
1028:['7', '10', '16', '22', '?']
1029:['3', '-1', '5', '1', '?']
1030:['10', '10', '8', '4', '?']
1031:['-7', '0', '1', '2', '9', '?

In [24]:
#converts all sequence items to float values
def floatConv(s):
    try:
        return float(s)
    except ValueError:
        try:
            num, denom = s.split('/')
            return float(num) / float(denom)
        except ValueError:
            return 'X'   #bad

#converts all the sequences to floats
def seq2Float(seq):
    floatSeq = {}
    for i in seq.keys():
        s = list(map(lambda x: floatConv(x) if x != "?" else x, seq[i]))
        if "X" in s:  #found a bad one (don't use)
            continue
        else:
            floatSeq[i] = s
    return floatSeq
print(seq2Float(gs))

{0: [7.0, 9.0, -1.0, 5.0, '?'], 1: [3.0, 2.0, 1.6666666666666667, 1.5, '?'], 2: [1.0, 2.0, 5.0, 26.0, '?'], 3: [2.0, 12.0, 30.0, '?'], 4: [2.0, 1.0, 0.6666666666666666, 0.5, '?'], 5: [4.0, 2.0, 2.0, 3.0, 6.0, '?'], 6: [1.0, 7.0, 8.0, 57.0, '?'], 7: [4.0, 12.0, 8.0, 10.0, '?'], 8: [0.5, 1.0, 1.0, '?', 0.8181818181818182, 0.8461538461538461], 9: [95.0, 88.0, 71.0, 61.0, 50.0, '?'], 10: [2.0, 6.0, 13.0, 39.0, 15.0, 45.0, 23.0, '?'], 11: [1.0, 3.0, 3.0, 5.0, 7.0, 9.0, 13.0, 15.0, '?', '?'], 12: [1.0, 2.0, 8.0, 28.0, '?'], 13: [0.0, 4.0, 18.0, '?', 100.0], 14: [1.0, 1.0, 2.0, 2.0, 3.0, 4.0, 3.0, 5.0, '?'], 15: [1.0, 52.0, 313.0, 174.0, '?'], 16: [5.0, 15.0, 10.0, 215.0, '?'], 17: [-7.0, 0.0, 1.0, 2.0, 9.0, '?'], 18: [0.0, 1.0, 3.0, 10.0, '?'], 19: [4.5, 14.0, 32.5, '?', 108.5], 20: [1.0, 1.0, 2.0, 6.0, 24.0, '?'], 21: [3.0, 4.0, 8.0, 24.0, 88.0, '?'], 22: [20.0, 22.0, 25.0, 30.0, 37.0, '?'], 23: [0.1111111111111111, 0.07407407407407407, 0.037037037037037035], 25: [1.0, 2.0, 4.0, 8.0, 16.0, 

## Classifier

In [25]:
valid = seq2Float(gs)  #good sequences dictionary
len(valid)

1033

In [26]:
# data_for_class = []
# for key in valid.keys():
#     data_for_class

In [27]:
maxx = 0
for key in valid.keys():
    if len(valid.get(key))>maxx:
        maxx = len(valid.get(key))
maxx

17

In [28]:
secuencias = []
clases = []
for key in valid.keys():
    if key in [82, 257, 298, 306, 451, 452, 463, 464,468, 476,495, 498,499, 500,504,505,623,624,626,627,628, 
               23, 501, 639, 697, 1056, 1073,
               11, 134, 158, 253, 467, 470, 475, 636, 769, 793, 816, 818, 833, 856, 862, 945]:
        continue


    secuencias.append(valid.get(key))
    clases.append(seqDataAns.classes[key])

In [29]:
secuencias

[[7.0, 9.0, -1.0, 5.0, '?'],
 [3.0, 2.0, 1.6666666666666667, 1.5, '?'],
 [1.0, 2.0, 5.0, 26.0, '?'],
 [2.0, 12.0, 30.0, '?'],
 [2.0, 1.0, 0.6666666666666666, 0.5, '?'],
 [4.0, 2.0, 2.0, 3.0, 6.0, '?'],
 [1.0, 7.0, 8.0, 57.0, '?'],
 [4.0, 12.0, 8.0, 10.0, '?'],
 [0.5, 1.0, 1.0, '?', 0.8181818181818182, 0.8461538461538461],
 [95.0, 88.0, 71.0, 61.0, 50.0, '?'],
 [2.0, 6.0, 13.0, 39.0, 15.0, 45.0, 23.0, '?'],
 [1.0, 2.0, 8.0, 28.0, '?'],
 [0.0, 4.0, 18.0, '?', 100.0],
 [1.0, 1.0, 2.0, 2.0, 3.0, 4.0, 3.0, 5.0, '?'],
 [1.0, 52.0, 313.0, 174.0, '?'],
 [5.0, 15.0, 10.0, 215.0, '?'],
 [-7.0, 0.0, 1.0, 2.0, 9.0, '?'],
 [0.0, 1.0, 3.0, 10.0, '?'],
 [4.5, 14.0, 32.5, '?', 108.5],
 [1.0, 1.0, 2.0, 6.0, 24.0, '?'],
 [3.0, 4.0, 8.0, 24.0, 88.0, '?'],
 [20.0, 22.0, 25.0, 30.0, 37.0, '?'],
 [1.0, 2.0, 4.0, 8.0, 16.0, '?'],
 [2.0, 1.0, 0.6666666666666666, 0.5, '?'],
 [1.0, 1.0, 3.0, 7.0, 17.0, 41.0, '?'],
 [2.5, 5.0, 12.5, 37.5, '?'],
 [6.0, 15.0, 35.0, 77.0, '?'],
 [1.0, 3.0, 3.0, 6.0, 7.0, 12.0, 15.0

In [30]:
question = [] #position of question mark
seq_len = []
for i in range(len(secuencias)):
    question.append(secuencias[i].index('?'))
    seq_len.append(len(secuencias[i]))
    


In [31]:
secuencias2 = secuencias[:] #replace '?' by zero
for i in range(len(secuencias2)):
    secuencias2[i][question[i]] = 0
    
    

In [32]:
secuencias3 = pad_sequences(secuencias2, padding='post') #make all sequences same lenght adding zeros

In [33]:
secuencias3 = secuencias3.tolist()


In [34]:
mask = []
for i in range(len(secuencias3)):
    mask.append([0] * len(secuencias3[0])) 

In [35]:
for i in range(len(mask)):
    for j in range(seq_len[i]):
        mask[i][j] = 1  #assign 1 in the mask sequence for actual values in sequence

In [36]:
for i in range(len(mask)):
    mask[i][question[i]] = 2 #assign 2 in the mask sequence for question mark position

In [37]:
#concatenate the sequence and the mask
secuencias4 = []
for i in range(len(secuencias3)):
    secuencias4.append(secuencias3[i]+mask[i])

In [38]:
len(clases)

993

## Classifier

In [39]:
#change question mark with '-1' to use the single serie
secuencias5 = secuencias3[:] 
for i in range(len(secuencias5)):
    secuencias3[i][question[i]] = -1


In [40]:
clases.index(1)

874

In [41]:
secuencias4.pop(clases.index(1));

In [42]:
clases.pop(clases.index(1))

1

In [74]:
#X_train, X_test, y_train, y_test = train_test_split(secuencias4, clases, test_size=0.15, stratify = clases)#, random_state=42) #train using the mask
X_train, X_test, y_train, y_test = train_test_split(secuencias4, clases, test_size=0.05, stratify = clases)#, random_state=42) #train using the mask
#X_train, X_test, y_train, y_test = train_test_split(secuencias5, clases, test_size=0.2, random_state=42) #train with just the serie


In [75]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [76]:
X_train = X_train.reshape((len(X_train), 34, 1))
X_test = X_test.reshape((len(X_test), 34,1))

#X_train = X_train.reshape((794, 17, 1))
#X_test = X_test.reshape((199, 17,1))


y_test = pd.get_dummies(y_test).values
y_train = pd.get_dummies(y_train).values


In [77]:
X_train.shape

(942, 34, 1)

In [78]:
y_train.shape

(942, 3)

In [79]:
model = Sequential()
#model.add(LSTM(100, input_shape= (34, 1)))
model.add(LSTM(100, input_shape= (34, 1)))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 500, batch_size = 8)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_6 (Dense)              (None, 32)                3232      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 99        
Total params: 44,131
Trainable params: 44,131
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
118/118 [==============================] - 1s 10ms/step - loss: 0.6014 - accuracy: 0.4682 - val_loss: 0.6011 - val_accuracy: 0.4800
Epoch 2/500
118/118 [==============================] - 1s 8ms/step - loss: 0.5962 - accuracy: 0.4841 - val_loss: 0.5955 - val_accuracy: 0.4800
Epoch 3/500
118/118 [==============================] - 1s 7ms/step - loss: 0.5950 - accu

Epoch 109/500
118/118 [==============================] - 1s 8ms/step - loss: 0.5795 - accuracy: 0.5085 - val_loss: 0.5985 - val_accuracy: 0.5400
Epoch 110/500
118/118 [==============================] - 1s 8ms/step - loss: 0.5822 - accuracy: 0.5085 - val_loss: 0.5710 - val_accuracy: 0.5600
Epoch 111/500
118/118 [==============================] - 1s 8ms/step - loss: 0.5784 - accuracy: 0.5042 - val_loss: 0.5759 - val_accuracy: 0.5400
Epoch 112/500
118/118 [==============================] - 1s 8ms/step - loss: 0.5773 - accuracy: 0.5106 - val_loss: 0.5949 - val_accuracy: 0.5600
Epoch 113/500
118/118 [==============================] - 1s 8ms/step - loss: 0.5737 - accuracy: 0.5138 - val_loss: 0.5770 - val_accuracy: 0.5800
Epoch 114/500
118/118 [==============================] - 1s 7ms/step - loss: 0.5803 - accuracy: 0.5096 - val_loss: 0.5780 - val_accuracy: 0.5400
Epoch 115/500
118/118 [==============================] - 1s 7ms/step - loss: 0.5754 - accuracy: 0.5085 - val_loss: 0.5765 - val_ac

118/118 [==============================] - 1s 10ms/step - loss: 0.5249 - accuracy: 0.5839 - val_loss: 0.7330 - val_accuracy: 0.5200
Epoch 222/500
118/118 [==============================] - 1s 11ms/step - loss: 0.5326 - accuracy: 0.5775 - val_loss: 0.7511 - val_accuracy: 0.4600
Epoch 223/500
118/118 [==============================] - 1s 11ms/step - loss: 0.5237 - accuracy: 0.5775 - val_loss: 0.7234 - val_accuracy: 0.4000
Epoch 224/500
118/118 [==============================] - 1s 8ms/step - loss: 0.5232 - accuracy: 0.5913 - val_loss: 0.7494 - val_accuracy: 0.4800
Epoch 225/500
118/118 [==============================] - 1s 8ms/step - loss: 0.5492 - accuracy: 0.5807 - val_loss: 0.7420 - val_accuracy: 0.4600
Epoch 226/500
118/118 [==============================] - 1s 8ms/step - loss: 0.5267 - accuracy: 0.5924 - val_loss: 0.7049 - val_accuracy: 0.4600
Epoch 227/500
118/118 [==============================] - 1s 8ms/step - loss: 0.5435 - accuracy: 0.5764 - val_loss: 0.7365 - val_accuracy: 0.4

118/118 [==============================] - 1s 8ms/step - loss: 0.4843 - accuracy: 0.6157 - val_loss: 0.8084 - val_accuracy: 0.4600
Epoch 334/500
118/118 [==============================] - 1s 8ms/step - loss: 0.4695 - accuracy: 0.6274 - val_loss: 0.8761 - val_accuracy: 0.5000
Epoch 335/500
118/118 [==============================] - 1s 8ms/step - loss: 0.4661 - accuracy: 0.6263 - val_loss: 0.9471 - val_accuracy: 0.4200
Epoch 336/500
118/118 [==============================] - 1s 8ms/step - loss: 0.4674 - accuracy: 0.6380 - val_loss: 0.9576 - val_accuracy: 0.4200
Epoch 337/500
118/118 [==============================] - 1s 9ms/step - loss: 0.4756 - accuracy: 0.6210 - val_loss: 0.8679 - val_accuracy: 0.4400
Epoch 338/500
118/118 [==============================] - 1s 9ms/step - loss: 0.4581 - accuracy: 0.6412 - val_loss: 0.8571 - val_accuracy: 0.4600
Epoch 339/500
118/118 [==============================] - 1s 8ms/step - loss: 0.4573 - accuracy: 0.6380 - val_loss: 0.9185 - val_accuracy: 0.4600

118/118 [==============================] - 1s 8ms/step - loss: 0.4394 - accuracy: 0.6476 - val_loss: 1.0071 - val_accuracy: 0.4400
Epoch 446/500
118/118 [==============================] - 1s 8ms/step - loss: 0.4256 - accuracy: 0.6561 - val_loss: 1.0984 - val_accuracy: 0.4600
Epoch 447/500
118/118 [==============================] - 1s 8ms/step - loss: 0.4353 - accuracy: 0.6401 - val_loss: 1.0890 - val_accuracy: 0.4200
Epoch 448/500
118/118 [==============================] - 1s 10ms/step - loss: 0.4417 - accuracy: 0.6497 - val_loss: 1.0874 - val_accuracy: 0.3800
Epoch 449/500
118/118 [==============================] - 1s 9ms/step - loss: 0.4259 - accuracy: 0.6603 - val_loss: 1.1493 - val_accuracy: 0.3400
Epoch 450/500
118/118 [==============================] - 1s 9ms/step - loss: 0.4363 - accuracy: 0.6539 - val_loss: 1.0594 - val_accuracy: 0.4000
Epoch 451/500
118/118 [==============================] - 1s 8ms/step - loss: 0.4306 - accuracy: 0.6539 - val_loss: 0.9693 - val_accuracy: 0.400

In [82]:
model.save("classifier")
#reconstructed_model = keras.models.load_model("classifier")

INFO:tensorflow:Assets written to: classifier/assets


In [93]:
import keras
reconstructed_model = keras.models.load_model("classifier")

In [94]:
X_train.shape

(942, 34, 1)

In [95]:
from keras.preprocessing.sequence import pad_sequences
def seq_prep(seq, max_len):
    question = [] #position of question mark
    seq_len = []
    for i in range(len(seq)):
        question.append(seq[i].index('?'))
        seq_len.append(len(seq[i]))
        seq[i][question[i]] = 0
    seq = pad_sequences(seq, padding='post', maxlen=max_len)
    seq = seq.tolist()
    mask = []
    for i in range(len(seq)):
        mask.append([0] * max_len)
    for i in range(len(mask)):
        for j in range(seq_len[i]):
            mask[i][j] = 1  #assign 1 in the mask sequence for actual values in sequence
    for i in range(len(mask)):
        mask[i][question[i]] = 2 #assign 2 in the mask sequence for question mark position
    seq2 = []
    for i in range(len(seq)):
        seq2.append(seq[i]+mask[i])
    return np.array([np.array(seq2).transpose()])

In [96]:
z = seq_prep([[1,1,2,3,5,'?']],17)
print(z.shape)

(1, 34, 1)


In [97]:
reconstructed_model.predict(z)

array([[0.04346835, 0.2678295 , 0.6887021 ]], dtype=float32)

In [98]:
np.array([X_train[0]]).shape

(1, 34, 1)

In [99]:
print(X_train[0])
print(y_train[0])

[[ 0]
 [11]
 [ 9]
 [ 9]
 [ 8]
 [ 7]
 [ 7]
 [ 5]
 [ 6]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 2]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]]
[0 0 1]


In [100]:
e = reconstructed_model.predict(np.array([X_train[0]]))

In [101]:
print(e)
print(np.argmax(e))

[[3.6821555e-13 1.5047198e-09 1.0000000e+00]]
2
